# pyMeccano v1.2
Scott D. Thiel (sthiel@umass.edu), Kimberly A. Bolduc (kbolduc@umass.edu), James. P. S. Walsh (jpswalsh@umass.edu)

Department of Chemistry, University of Massachusetts Amherst

The MIT License (MIT)
Copyright © 2021 Scott D. Thiel, Kimberly A. Bolduc, James. P. S. Walsh

---

## What is this?
This is a Jupyter Notebook script to handle the integration of diffraction images collected at the Matter in Extreme Conditions beamline at SLAC. It uses pyFAI to integrate multiple detector images into a single 1D lineout. It can be set to run automatically, integrating images as they appear in the scratch directory.

---

## Getting started
* Copy this `.ipynb` file to your `psana/` user directory on Confluence, or into a subdirectory.
* Set the Jupyter Notebook kernel to one that contains all the required packages. This notebook was devloped and tested using `anaplus1`, which was set up by Scott Thiel. This can be added with the following commands:

    ```
    mkdir -p ~/.local/share/jupyter/kernels/anaplus1
    cp ~sthiel/.local/share/jupyter/kernels/anaplus1/kernel.json ~/.local/share/jupyter/kernels/anaplus1/kernel.json
    ```

    However, using this kernel is implicitly trusting Scott and the code in the kernel. If you prefer to recreate a kernel/environment yourself, you can follow [this guide](https://docs.google.com/document/d/1GRGpmNkRPROySZmbzEolMHFaOeUfFk6Edaifp9f3DRw/edit?usp=sharing), or reach out to sthiel@umass.edu for help.
* If you want to use masks in your integrations, then add these to the `psana/mask/` directory. Check the `mask_form` in the **Variables** section for the expected naming convention of these files.
* Add your `.poni` calibration files to `psana/poni{day}/`, where `day` is the day of the calibration. This is useful for when the calibration changes day-to-day. Note that it doesn't necessarily need to be a day, just a unique number that the `date_rule` function can map to.
* Run the **Imports** cell. You can ignore the warning about pyOpenCl.
* Set the experiment variables in the **Variables** cell and run this cell.
* Change the `date_rule` function in the **Functions** cell to an appropriate rule for poni locations and run this cell.
* Run the **Widgets** cell.
* Set the `run_range` in the **Integration** cell. Set a high number to run in automatic mode—it will wait for new images.
* Run the **Integration** cell.
* Informaiton regarding the status of integrations will be displayed in boxes under the cell.

---

## Expected file structure

```
.
└── psana/ <-- Or whatever subdirectory you choose.
    ├── pyMeccano_v1.2.ipynb
    ├── mask/
    ├── poni{day}/
    ├── poni{day}/
    └── out/
        ├── run{run_number}_evt{event}_poni{date}_masked-allquad-lineout.xy
        ├── run{run_number}_evt{event}_poni{date}_masked-allquad-lineout.png
        └── run{run_number}_evt{event}_poni{date}_masked-allquad-cake.png
```


## Imports
Run this cell first. You shouldn't have to change anything here.

In [ ]:
import os
import time
import shutil
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import fabio
import imageio
import psana
import scipy
from scipy import ndimage
import pyFAI
from pyFAI.gui import jupyter
from pyFAI import multi_geometry
import ipywidgets as widgets
from IPython.display import display

## Variables
Set the variables of your experiment here.

In [ ]:
# pyMeccano version tag name; will affect auto copy location
vtag = 'foxtrot'

# Set the experiment name as it appears in /reg/d/psdm/mec/
experiment = 'meclv1318'

# Set the default event and dark numbers to grab. The shot is usually _evt1, but you can change this if needed.
event = 1
max_event = 2
dark_event = 1

# Location and naming format for the mask files. 
# Mask should be applied to the original (unrotated) images.
mask_path = 'mask'
mask_form = 'quad{q:d}_sample-mask-new.edf'

# Rotation sense of the detectors (Q0, Q1, Q2, Q3) used in the .poni files.
quad_rotations = (-90, 90, 90, -90)

# runs with more than one event but the first event isn't a dark (like calibration runs)
not_darks = [
    29, 30, 31, 
    162, 163, 164, 165, 
    322, 323, 324, 325, 
    389, 390, 391, 392
]

# polarization correction between -1 (vertical) and +1 (horizontal) with 0 being circular or random
polarization_factor = -1

## Functions
May be changed to fit circumstances.

In [ ]:
# ========= CHANGE ME ===========
def date_rule(run_number):
    """
    Used to get different poni files depending on the date based on run.
    Change this function to fit circumstances. 
    """
    if run_number > 319:
        return 22
    elif run_number > 160:
        return 21
    else:
        return 20
# ===============================

def find_tiff(quad=0, run=1, event=1, exp=experiment, where='scratch'):
    # assumes the Epix10ka detector
    hutch = exp[0:3]
    mec_root = f'/reg/d/psdm/{hutch}/{exp}/{where}'
    tiff_file = f'Run_{run:d}_evt_{event:d}_Epix10kaQuad{quad:d}.tiff'
    tiff_path = os.path.join(mec_root, f'run{run:d}', tiff_file)
    return tiff_path

def get_image(quad=0, run=1, event=1, exp=experiment, where='scratch'):
    tiff = find_tiff(quad, run, event, exp, where)
    img = fabio.open(tiff).data
    return img

def num_events_in_run(run):
    """
    Count events using quad0 as reference.
    If there are 5 files with `Quad0` in the name, this will count that as 5 events.
    """
    run_dir = os.path.split(find_tiff(run=run))[0]
    count = len([entry for entry in os.scandir(run_dir) if entry.is_file() and 'Quad0' in entry.name])
    return count

def is_shot_run(run, n_events=1):
    """Shot runs should have `n_events` number of events."""
    return num_events_in_run(run) == n_events

def is_dark_run(run, not_darks=None):
    """Assume if it is a dark if it is not a shot unless otherwise specified in `not_darks`."""
    if not_darks and run in not_darks:
        return False
    return not is_shot_run(run)

def find_dark_run(run, not_darks=None):
    """
    If the run is a shot, find the last pre-shot run. Otherwise it is a pre-shot already.
    Runs in `not_darks` are treated as not dark runs as in `is_dark_run` function.
    """
    p_run = run
    while not is_dark_run(p_run, not_darks):
        p_run -= 1
    return p_run

def display_info(widget, output, value):
    """
    Display new info for the dashboard.
    """
    with output:
        widget.value = str(value)
        output.clear_output()
        display(widget)
    return


#### Widgets
For the boxes of info. Cosmetic.

In [ ]:
capture_plots = widgets.Output()
last_lineout = widgets.Output()
layout = widgets.Layout(align_items='center', border='solid black', padding='0 0 0 30px', width='25%')
layout2 = widgets.Layout(align_items='center', border='solid black', padding='0 0 0 30px', width='75%')
layout3 = widgets.Layout(align_items='center')
layout4 = widgets.Layout(align_items='center', border='solid black', padding='0 0 0 30px', width='50%')
run_num_l = widgets.Label('Active Run:')
run_num_w = widgets.Label('n/a')
run_num_o = widgets.Output()
evt_num_l = widgets.Label('Event:')
evt_num_w = widgets.Label('n/a')
evt_num_o = widgets.Output()
run_status_l = widgets.Label('Status:')
run_status_w = widgets.Label('n/a')
run_status_o = widgets.Output()
run_info_l = widgets.Label('Info:')
run_info_w = widgets.Label('n/a')
run_info_o = widgets.Output()
run_info_b = widgets.HBox([run_info_l, run_info_o], layout=layout2)
run_num_b = widgets.HBox([run_num_l, run_num_o], layout=layout3)
evt_num_b = widgets.HBox([evt_num_l, evt_num_o], layout=layout3)
run_status_b = widgets.HBox([run_status_l, run_status_o], layout=layout4)
run_evt_num_b = widgets.HBox([run_num_b, evt_num_b], layout=layout)
run_row_b = widgets.HBox([run_evt_num_b, run_status_b])
dashboard = widgets.VBox([run_row_b, run_info_b])
with run_num_o:
    display(run_num_w)
with run_status_o:
    display(run_status_w)
with evt_num_o:
    display(evt_num_w)
with run_info_o:
    display(run_info_w)

## Integration

In [ ]:
%matplotlib inline
display(dashboard)
display(last_lineout)
display(capture_plots)

# Set the range of runs to analyze. The first number is inclusive, the second number is exclusive. E.g. (30,34) will analyze 30+31+32+33.
# When using in automatic mode, just pick a high second number that won't be reached, like 1000.
run_range = range(30, 31)

# Flags

subtract_darks = True  # <- Whether to subtract darks

display_last = True  # <-- Whether to display the last 1d integration

use_not_darks = True # <- Whether to use the not_darks list

# Whether to automatically copy to the `scratch/pymeccano-{vtag}` folder in experiment location
# Unecessary when already running this somewhere in the scratch folder.
auto_copy_to_exp_folder = False

for run_number in run_range:
    
    display_info(run_num_w, run_num_o, run_number)
    
    while not os.path.exists(os.path.split(find_tiff(run=run_number))[0]):
        display_info(run_status_w, run_status_o, 'waiting for run')
        time.sleep(5)
    
    display_info(run_status_w, run_status_o, 'scanning events')
    dark_run = find_dark_run(run_number, not_darks if use_not_darks else None)
    events = num_events_in_run(run_number)
    events = min(max_event, events)
    event_range = range(1, events + 1)
    
    if subtract_darks:
        display_info(run_info_w, run_info_o, f'using run{dark_run}_evt{dark_event} as dark')
    else:
        display_info(run_info_w, run_info_o, 'not applying darks')
    
    for event in event_range:
        
        display_info(evt_num_w, evt_num_o, event)
        
        loaded = False
        # load in images and darks
        display_info(run_status_w, run_status_o, 'loading images')
        try:
            imgs_raw = [get_image(quad=q, run=run_number, event=event, exp=experiment) for q in range(4)]
            darks_raw = [get_image(quad=q, run=dark_run, event=dark_event, exp=experiment) for q in range(4)]
            loaded = True
        except:
            display_info(run_status_w, run_status_o, 'error loading')
            break
        
        # rotate images
        imgs_fup = [np.flipud(img) for img in imgs_raw]
        imgs_ll_orient_for_poni = [ndimage.rotate(imgs_fup[q], quad_rotations[q]) for q in range(4)]
        imgs = imgs_ll_orient_for_poni

        # rotate darks
        darks_fup = [np.flipud(img) for img in darks_raw]
        darks_ll_orient_for_poni = [ndimage.rotate(darks_fup[q], quad_rotations[q]) for q in range(4)]
        darks = darks_ll_orient_for_poni

        # load in and rotate masks
        quad_masks = [fabio.open(os.path.join(mask_path, mask_form.format(q=q))).data for q in range(4)]
        qmasks_fup = [np.flipud(qmask) if qmask is not None else None for qmask in quad_masks]
        qmasks_ll_orient_for_poni = [
            ndimage.rotate(qmasks_fup[q], quad_rotations[q]) if qmasks_fup[q] is not None else None 
            for q in range(4)
        ]
        qmasks = qmasks_ll_orient_for_poni

        # location and naming of the poni files 
        date = date_rule(run_number)
        poni_path = f'poni{date}'
        poni_form = 'quad{q:d}-rotated.poni'
        
        display_info(run_status_w, run_status_o, 'integrating')

        # load individual detector geometries
        azis = [pyFAI.load(os.path.join(poni_path, poni_form.format(q=q))) for q in range(4)]

        # this is the data to use for integration
        lst_data = [(imgs[i] - darks[i] if subtract_darks else imgs[i]) for i in range(4)]

        # integrate all detectors together
        mazi = multi_geometry.MultiGeometry(azis, radial_range=(0, 80), azimuth_range=(-180, 180), chi_disc=180 )
        mres1 = mazi.integrate1d(lst_data, lst_mask=qmasks, polarization_factor=polarization_factor)
        mresdark = mazi.integrate1d(darks)
        mres2 = mazi.integrate2d(lst_data, lst_mask=qmasks, polarization_factor=polarization_factor)
        mres1_arr = np.array(mres1)

        # this can be changed to change the output folder
        outf = 'out' if subtract_darks else 'out_nodark'
        pathlib.Path(outf).mkdir(exist_ok=True, parents=True)

        ln_xy = f'{outf}/run{run_number}_evt{event}_poni{date}_masked-allquad-lineout.xy'
        ln_png = f'{outf}/run{run_number}_evt{event}_poni{date}_masked-allquad-lineout.png'
        cake_png = f'{outf}/run{run_number}_evt{event}_poni{date}_masked-allquad-cake.png'

        display_info(run_status_w, run_status_o, 'writing')
        
        # save an .xy output
        np.savetxt(ln_xy, mres1_arr.T)
        
        with last_lineout:
            last_lineout.clear_output()
            fig, ax = plt.subplots()
            jupyter.plot1d(mres1, label=f'run{run_number}_evt{event}', ax=ax)
            ax.set_xlim([15, 70])
            fig.savefig(ln_png)
            if display_last:
                display(fig)
            plt.close(fig)
        with capture_plots:
            capture_plots.clear_output()
            fig, ax = plt.subplots()
            jupyter.plot2d(mres2, ax=ax)
            ax.set_ylim([50, 180])
            fig.savefig(cake_png)
            plt.close(fig)
        if auto_copy_to_exp_folder:
            pathlib.Path(f"/reg/d/psdm/{experiment[0:3]}/{experiment}/scratch/pymeccano-{vtag}").mkdir(exist_ok=True, parents=True)
            shutil.copy2(ln_xy, f"/reg/d/psdm/{experiment[0:3]}/{experiment}/scratch/pymeccano-{vtag}/{os.path.basename(ln_xy)}")
            shutil.copy2(ln_png, f"/reg/d/psdm/{experiment[0:3]}/{experiment}/scratch/pymeccano-{vtag}/{os.path.basename(ln_png)}")
            shutil.copy2(cake_png, f"/reg/d/psdm/{experiment[0:3]}/{experiment}/scratch/pymeccano-{vtag}/{os.path.basename(cake_png)}")
        display_info(run_status_w, run_status_o, 'done')